In [1]:
import numpy as np 
import pandas as pd 
from data_processing import generate_vocab, process_data, create_dataloaders, text2tokens
from model import get_pretrained_emb, EncoderDecoder, EncoderRNN, DecoderRNN, EncoderDecoderAttn, DecoderAttnRNN
from train_eval import count_parameters, summarize_results, plot_single_learning_curve, load_experiment_log
from train_eval import train_and_eval
import importlib
import pickle as pkl 
import torch

In [2]:
# model identification
MODEL_NAME = 'zh-seq2seq-rnn-vanilla'
SRC_LANG = 'zh'
TARG_LANG = 'en'

# data processing params  
SRC_MAX_SENTENCE_LEN = 10
TARG_MAX_SENTENCE_LEN = 10
SRC_VOCAB_SIZE = 30000 #30000
TARG_VOCAB_SIZE = 30000 #30000

# model architecture params 
RNN_CELL_TYPE = 'gru'
NUM_LAYERS = 2 #2 
ENC_HIDDEN_DIM = 256 #512
DEC_HIDDEN_DIM = 2 * ENC_HIDDEN_DIM #2 * ENC_HIDDEN_DIM 
TEACHER_FORCING_RATIO = 1
CLIP_GRAD_MAX_NORM = 1
ENC_DROPOUT = 0.2 # to actually implement
DEC_DROPOUT = 0.2 # to actually implement
USE_ATTN = True

# training params  
BATCH_SIZE = 32 #32
NUM_EPOCHS = 200
LR = 0.0003 # 0.0005
OPTIMIZER = 'Adam'
LAZY_TRAIN = True

In [3]:
# store as dict to save to results later 
params = {'model_name': MODEL_NAME, 'src_lang': SRC_LANG, 'targ_lang': TARG_LANG, 'rnn_cell_type': RNN_CELL_TYPE, 
          'src_max_sentence_len': SRC_MAX_SENTENCE_LEN, 'targ_max_sentence_len': TARG_MAX_SENTENCE_LEN, 
          'src_vocab_size': SRC_VOCAB_SIZE, 'targ_vocab_size': TARG_VOCAB_SIZE, 
          'num_layers': NUM_LAYERS, 'enc_hidden_dim': ENC_HIDDEN_DIM, 'dec_hidden_dim': DEC_HIDDEN_DIM,
          'teacher_forcing_ratio': TEACHER_FORCING_RATIO, 'clip_grad_max_norm': CLIP_GRAD_MAX_NORM,
          'enc_dropout': ENC_DROPOUT, 'dec_dropout': DEC_DROPOUT, 'use_attn': USE_ATTN, 
          'batch_size': BATCH_SIZE, 'num_epochs': NUM_EPOCHS, 'learning_rate': LR, 'optimizer': OPTIMIZER, 
          'lazy_train': LAZY_TRAIN} 

In [4]:
# # takes a long time to process, save to pickle for reimport in future 
# vocab = generate_vocab(SRC_LANG, TARG_LANG, SRC_VOCAB_SIZE, TARG_VOCAB_SIZE)
# vocab_filename = "{}-{}-vocab.p".format(SRC_LANG, TARG_LANG)
# pkl.dump(vocab, open(vocab_filename, "wb"))

In [5]:
# reload from pickle 
vocab_filename = "{}-{}-vocab.p".format(SRC_LANG, TARG_LANG)
vocab = pkl.load(open(vocab_filename, "rb"))
data = process_data(SRC_LANG, TARG_LANG, SRC_MAX_SENTENCE_LEN, TARG_MAX_SENTENCE_LEN, vocab)
data_minibatch = process_data(SRC_LANG, TARG_LANG, SRC_MAX_SENTENCE_LEN, TARG_MAX_SENTENCE_LEN, vocab, sample_limit=BATCH_SIZE) 
data_minitrain = process_data(SRC_LANG, TARG_LANG, SRC_MAX_SENTENCE_LEN, TARG_MAX_SENTENCE_LEN, vocab, sample_limit=1000)

In [6]:
# create dataloaders 
loaders_full = create_dataloaders(data, SRC_MAX_SENTENCE_LEN, TARG_MAX_SENTENCE_LEN, BATCH_SIZE)
loaders_minibatch = create_dataloaders(data_minibatch, SRC_MAX_SENTENCE_LEN, TARG_MAX_SENTENCE_LEN, BATCH_SIZE)
loaders_minitrain = create_dataloaders(data_minitrain, SRC_MAX_SENTENCE_LEN, TARG_MAX_SENTENCE_LEN, BATCH_SIZE)

In [7]:
# define model 

encoder = EncoderRNN(rnn_cell_type=RNN_CELL_TYPE, enc_hidden_dim=ENC_HIDDEN_DIM, num_layers=NUM_LAYERS, 
                     src_max_sentence_len=SRC_MAX_SENTENCE_LEN, enc_dropout=ENC_DROPOUT, 
                     pretrained_word2vec=get_pretrained_emb(vocab[SRC_LANG]['word2vec'], vocab[SRC_LANG]['token2id']))

# without attention 
# decoder = DecoderRNN(dec_hidden_dim=DEC_HIDDEN_DIM, enc_hidden_dim=ENC_HIDDEN_DIM, num_layers=NUM_LAYERS,
#                      targ_vocab_size=TARG_VOCAB_SIZE, targ_max_sentence_len=TARG_MAX_SENTENCE_LEN, 
#                      pretrained_word2vec=get_pretrained_emb(vocab[TARG_LANG]['word2vec'], vocab[TARG_LANG]['token2id']))
# model = EncoderDecoder(encoder, decoder, vocab[TARG_LANG]['token2id'])

# with attention 
decoder = DecoderAttnRNN(rnn_cell_type=RNN_CELL_TYPE, dec_hidden_dim=DEC_HIDDEN_DIM, enc_hidden_dim=ENC_HIDDEN_DIM, 
                         num_layers=NUM_LAYERS, targ_vocab_size=TARG_VOCAB_SIZE, src_max_sentence_len=SRC_MAX_SENTENCE_LEN, 
                         targ_max_sentence_len=TARG_MAX_SENTENCE_LEN, dec_dropout=DEC_DROPOUT, 
                         pretrained_word2vec=get_pretrained_emb(vocab[TARG_LANG]['word2vec'], vocab[TARG_LANG]['token2id']))
model = EncoderDecoderAttn(encoder, decoder, vocab[TARG_LANG]['token2id']) 

In [ ]:
model, results = train_and_eval(
    model=model, loaders_full=loaders_full, loaders_minibatch=loaders_minibatch, loaders_minitrain=loaders_minitrain, 
    params=params, vocab=vocab, print_intermediate=True, save_checkpoint=True, save_to_log=True, 
    lazy_eval=False, print_attn=True, inspect_samples=1)

Epoch: 0.00, Train Loss: 9.89, Val Loss: 10.26, Train BLEU: 0.01, Val BLEU: 0.10, Minutes Elapsed: 0.06
Sampling from training predictions...
Source: 它们 被叫 叫做 管道 蠕虫 <EOS> <PAD> <PAD> <PAD> <PAD>
Reference: these are called tube worms . <EOS> <PAD> <PAD>
Model: <SOS> yukon <EOS> <EOS> <EOS> <EOS> <EOS> <EOS> <EOS> <EOS>
Attention Weights: tensor([[0.0024, 0.0025, 0.0025, 0.0026, 0.0026, 0.0026, 0.2462, 0.2462, 0.2462,
         0.2462],
        [0.0024, 0.0024, 0.0025, 0.0026, 0.0026, 0.0026, 0.2462, 0.2462, 0.2462,
         0.2462],
        [0.0024, 0.0024, 0.0025, 0.0026, 0.0026, 0.0025, 0.2462, 0.2462, 0.2462,
         0.2462],
        [0.0024, 0.0025, 0.0025, 0.0026, 0.0026, 0.0026, 0.2462, 0.2462, 0.2462,
         0.2462],
        [0.0024, 0.0025, 0.0025, 0.0026, 0.0026, 0.0026, 0.2462, 0.2462, 0.2462,
         0.2462],
        [0.0024, 0.0025, 0.0026, 0.0026, 0.0026, 0.0026, 0.2462, 0.2462, 0.2462,
         0.2462],
        [0.0024, 0.0025, 0.0026, 0.0026, 0.0027, 0.0026, 0.2461, 0

Epoch: 4.00, Train Loss: 8.51, Val Loss: 10.02, Train BLEU: 6.74, Val BLEU: 1.26, Minutes Elapsed: 0.31
Sampling from training predictions...
Source: 海洋 的 平均 深度 是 两英里 英里 <EOS> <PAD> <PAD>
Reference: the average depth is about two miles . <EOS>
Model: <SOS> &apos;s <EOS> <EOS> <EOS> <EOS> <EOS> <EOS> <EOS> <EOS>
Attention Weights: tensor([[1.0443e-06, 1.1413e-06, 1.2228e-06, 1.2769e-06, 1.3129e-06, 1.2846e-06,
         1.3633e-06, 1.3385e-06, 5.0000e-01, 5.0000e-01],
        [7.8016e-07, 8.5521e-07, 9.1709e-07, 9.5791e-07, 9.8453e-07, 9.6118e-07,
         1.0207e-06, 1.0009e-06, 5.0000e-01, 5.0000e-01],
        [6.7296e-07, 7.3875e-07, 7.9235e-07, 8.2773e-07, 8.5061e-07, 8.2943e-07,
         8.8035e-07, 8.6351e-07, 5.0000e-01, 5.0000e-01],
        [6.2724e-07, 6.8903e-07, 7.3906e-07, 7.7226e-07, 7.9354e-07, 7.7297e-07,
         8.1940e-07, 8.0376e-07, 5.0000e-01, 5.0000e-01],
        [6.0916e-07, 6.6911e-07, 7.1756e-07, 7.4986e-07, 7.7041e-07, 7.5011e-07,
         7.9421e-07, 7.7893e-07

Epoch: 7.00, Train Loss: 7.49, Val Loss: 9.74, Train BLEU: 1.83, Val BLEU: 1.22, Minutes Elapsed: 0.51
Sampling from training predictions...
Source: 学习 掌握 它 <EOS> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Reference: learn to manage it . <EOS> <PAD> <PAD> <PAD>
Model: <SOS> . <EOS> <EOS> <EOS> <EOS> <EOS> <EOS> <EOS> <EOS>
Attention Weights: tensor([[3.8937e-08, 4.8224e-08, 5.3745e-08, 4.6473e-08, 1.6667e-01, 1.6667e-01,
         1.6667e-01, 1.6667e-01, 1.6667e-01, 1.6667e-01],
        [2.1736e-08, 2.7028e-08, 3.0253e-08, 2.6138e-08, 1.6667e-01, 1.6667e-01,
         1.6667e-01, 1.6667e-01, 1.6667e-01, 1.6667e-01],
        [1.7319e-08, 2.1511e-08, 2.4111e-08, 2.0907e-08, 1.6667e-01, 1.6667e-01,
         1.6667e-01, 1.6667e-01, 1.6667e-01, 1.6667e-01],
        [1.5947e-08, 1.9763e-08, 2.2130e-08, 1.9217e-08, 1.6667e-01, 1.6667e-01,
         1.6667e-01, 1.6667e-01, 1.6667e-01, 1.6667e-01],
        [1.5605e-08, 1.9295e-08, 2.1575e-08, 1.8746e-08, 1.6667e-01, 1.6667e-01,
         1.6667e-01, 1.666

Epoch: 10.00, Train Loss: 6.56, Val Loss: 9.41, Train BLEU: 0.63, Val BLEU: 0.92, Minutes Elapsed: 0.69
Sampling from training predictions...
Source: 这 是 一只 水母 <EOS> <PAD> <PAD> <PAD> <PAD> <PAD>
Reference: here &apos;s a jelly . <EOS> <PAD> <PAD> <PAD>
Model: <SOS> . . <EOS> <EOS> <EOS> <EOS> <EOS> <EOS> <EOS>
Attention Weights: tensor([[1.7553e-08, 2.8826e-08, 3.5360e-08, 3.5067e-08, 2.2030e-08, 2.0000e-01,
         2.0000e-01, 2.0000e-01, 2.0000e-01, 2.0000e-01],
        [8.1361e-09, 1.3557e-08, 1.6733e-08, 1.6585e-08, 1.0353e-08, 2.0000e-01,
         2.0000e-01, 2.0000e-01, 2.0000e-01, 2.0000e-01],
        [6.3057e-09, 1.0514e-08, 1.2995e-08, 1.2896e-08, 8.0876e-09, 2.0000e-01,
         2.0000e-01, 2.0000e-01, 2.0000e-01, 2.0000e-01],
        [5.7915e-09, 9.6445e-09, 1.1922e-08, 1.1832e-08, 7.4404e-09, 2.0000e-01,
         2.0000e-01, 2.0000e-01, 2.0000e-01, 2.0000e-01],
        [5.6993e-09, 9.4725e-09, 1.1706e-08, 1.1617e-08, 7.3227e-09, 2.0000e-01,
         2.0000e-01, 2.0000e-01

Epoch: 13.00, Train Loss: 5.77, Val Loss: 9.19, Train BLEU: 0.57, Val BLEU: 0.98, Minutes Elapsed: 0.88
Sampling from training predictions...
Source: 这 是 一只 水母 <EOS> <PAD> <PAD> <PAD> <PAD> <PAD>
Reference: here &apos;s a jelly . <EOS> <PAD> <PAD> <PAD>
Model: <SOS> . . . <EOS> <EOS> <EOS> <EOS> <EOS> <EOS>
Attention Weights: tensor([[1.1815e-08, 2.4382e-08, 3.1677e-08, 2.9625e-08, 1.4330e-08, 2.0000e-01,
         2.0000e-01, 2.0000e-01, 2.0000e-01, 2.0000e-01],
        [4.6656e-09, 9.9761e-09, 1.3153e-08, 1.2219e-08, 5.7420e-09, 2.0000e-01,
         2.0000e-01, 2.0000e-01, 2.0000e-01, 2.0000e-01],
        [3.6959e-09, 7.9748e-09, 1.0560e-08, 9.8049e-09, 4.5941e-09, 2.0000e-01,
         2.0000e-01, 2.0000e-01, 2.0000e-01, 2.0000e-01],
        [3.4490e-09, 7.4728e-09, 9.9188e-09, 9.2077e-09, 4.3139e-09, 2.0000e-01,
         2.0000e-01, 2.0000e-01, 2.0000e-01, 2.0000e-01],
        [3.3889e-09, 7.3467e-09, 9.7582e-09, 9.0587e-09, 4.2507e-09, 2.0000e-01,
         2.0000e-01, 2.0000e-01, 2.

...

Epoch: 169.00, Train Loss: 1.52, Val Loss: 12.52, Train BLEU: 35.64, Val BLEU: 0.26, Minutes Elapsed: 10.10
Sampling from training predictions...
Source: 它 是否 是否是 很 敏感 呢 是 的 <EOS> <PAD>
Reference: is it sensitive ? yes . <EOS> <PAD> <PAD>
Model: <SOS> is &apos;s sensitive of . . <EOS> <EOS> <EOS>
Attention Weights: tensor([[1.9461e-07, 1.0478e-06, 1.3903e-06, 1.6492e-06, 6.7878e-07, 2.3586e-07,
         8.7453e-08, 3.4311e-08, 1.7367e-08, 9.9999e-01],
        [3.3627e-07, 1.5763e-06, 2.0098e-06, 2.4538e-06, 1.0610e-06, 3.8455e-07,
         1.5462e-07, 6.4432e-08, 3.2843e-08, 9.9999e-01],
        [3.0168e-07, 1.3001e-06, 1.5935e-06, 1.9412e-06, 8.8356e-07, 3.5903e-07,
         1.6181e-07, 7.3414e-08, 3.8970e-08, 9.9999e-01],
        [2.2357e-07, 7.6937e-07, 8.6575e-07, 1.0829e-06, 6.1716e-07, 3.3550e-07,
         1.9519e-07, 1.0807e-07, 6.2718e-08, 1.0000e+00],
        [1.4634e-07, 3.6732e-07, 3.9983e-07, 5.1891e-07, 4.4494e-07, 3.7875e-07,
         3.2069e-07, 2.4441e-07, 1.7816e-07, 1

Epoch: 172.00, Train Loss: 1.48, Val Loss: 12.57, Train BLEU: 36.74, Val BLEU: 0.28, Minutes Elapsed: 10.26
Sampling from training predictions...
Source: 非常 非常感谢 感谢 <EOS> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Reference: thank you very much . <EOS> <PAD> <PAD> <PAD>
Model: <SOS> thank you very much . <EOS> <EOS> <EOS> <EOS>
Attention Weights: tensor([[1.2988e-09, 1.4284e-09, 6.0684e-09, 9.2807e-09, 1.6667e-01, 1.6667e-01,
         1.6667e-01, 1.6667e-01, 1.6667e-01, 1.6667e-01],
        [1.1342e-10, 9.3305e-11, 3.5932e-10, 5.4093e-10, 1.6667e-01, 1.6667e-01,
         1.6667e-01, 1.6667e-01, 1.6667e-01, 1.6667e-01],
        [7.4417e-11, 5.5983e-11, 1.9477e-10, 2.7895e-10, 1.6667e-01, 1.6667e-01,
         1.6667e-01, 1.6667e-01, 1.6667e-01, 1.6667e-01],
        [1.6749e-10, 1.2502e-10, 5.0541e-10, 7.3022e-10, 1.6667e-01, 1.6667e-01,
         1.6667e-01, 1.6667e-01, 1.6667e-01, 1.6667e-01],
        [1.8810e-09, 2.0507e-09, 9.4859e-09, 1.5074e-08, 1.6667e-01, 1.6667e-01,
         1.6667e-01, 

Epoch: 175.00, Train Loss: 1.44, Val Loss: 12.64, Train BLEU: 37.83, Val BLEU: 0.26, Minutes Elapsed: 10.42
Sampling from training predictions...
Source: 这 是 这些 热液 出口 口中 的 一个 <EOS> <PAD>
Reference: this is one of these hydrothermal vents . <EOS>
Model: <SOS> this is the is is hydrothermal . . <EOS>
Attention Weights: tensor([[2.7149e-07, 1.1793e-06, 1.8169e-06, 1.5689e-06, 4.3751e-07, 1.2875e-07,
         4.6611e-08, 1.8119e-08, 9.5667e-09, 9.9999e-01],
        [3.9913e-07, 1.6251e-06, 2.5401e-06, 2.2539e-06, 6.9170e-07, 2.1939e-07,
         8.1977e-08, 3.2672e-08, 1.7169e-08, 9.9999e-01],
        [3.7927e-07, 1.4158e-06, 2.1343e-06, 1.8859e-06, 6.4431e-07, 2.3479e-07,
         9.9166e-08, 4.2540e-08, 2.3120e-08, 9.9999e-01],
        [3.1509e-07, 1.0036e-06, 1.3796e-06, 1.1991e-06, 4.9970e-07, 2.3722e-07,
         1.2569e-07, 6.4255e-08, 3.7287e-08, 1.0000e+00],
        [2.3477e-07, 6.1681e-07, 7.8897e-07, 7.0800e-07, 4.0540e-07, 2.5943e-07,
         1.7480e-07, 1.0957e-07, 7.0003e-08,

Epoch: 178.00, Train Loss: 1.38, Val Loss: 12.66, Train BLEU: 44.20, Val BLEU: 0.26, Minutes Elapsed: 10.59
Sampling from training predictions...
Source: 这些 更 小 地动 动物 会 蜷缩 缩在 周围 <EOS>
Reference: there are smaller animals crawling around . <EOS> <PAD>
Model: <SOS> there are smaller animals crawling around . <EOS> <EOS>
Attention Weights: tensor([[0.0553, 0.1403, 0.3091, 0.3902, 0.0751, 0.0169, 0.0067, 0.0022, 0.0024,
         0.0019],
        [0.0447, 0.1184, 0.2861, 0.4332, 0.0812, 0.0199, 0.0085, 0.0027, 0.0029,
         0.0023],
        [0.0402, 0.1159, 0.2943, 0.4400, 0.0776, 0.0180, 0.0074, 0.0023, 0.0024,
         0.0019],
        [0.0527, 0.1365, 0.3103, 0.4011, 0.0702, 0.0160, 0.0067, 0.0023, 0.0024,
         0.0019],
        [0.0434, 0.1298, 0.2886, 0.3880, 0.0922, 0.0291, 0.0142, 0.0048, 0.0055,
         0.0044],
        [0.0657, 0.1448, 0.2828, 0.3583, 0.0831, 0.0279, 0.0156, 0.0070, 0.0080,
         0.0068],
        [0.0032, 0.0050, 0.0079, 0.0130, 0.0112, 0.0228, 0.0585, 0.

Epoch: 181.00, Train Loss: 1.33, Val Loss: 12.71, Train BLEU: 44.55, Val BLEU: 0.27, Minutes Elapsed: 10.76
Sampling from training predictions...
Source: 看到 这些 在 动 的 东西 了 吗 <EOS> <PAD>
Reference: but see all those different working things ? <EOS>
Model: <SOS> but see the see different ocean . . <EOS>
Attention Weights: tensor([[2.0771e-07, 8.9360e-07, 1.7339e-06, 1.5293e-06, 4.8197e-07, 1.6177e-07,
         4.8383e-08, 1.6944e-08, 8.9076e-09, 9.9999e-01],
        [3.1845e-07, 1.3108e-06, 2.6354e-06, 2.4812e-06, 8.0059e-07, 2.7805e-07,
         8.3298e-08, 2.9202e-08, 1.5296e-08, 9.9999e-01],
        [3.4960e-07, 1.3085e-06, 2.5869e-06, 2.5063e-06, 8.7023e-07, 3.3407e-07,
         1.1283e-07, 4.2497e-08, 2.2466e-08, 9.9999e-01],
        [2.8446e-07, 9.3048e-07, 1.5722e-06, 1.4294e-06, 5.7649e-07, 2.7724e-07,
         1.1962e-07, 5.3756e-08, 3.0594e-08, 9.9999e-01],
        [2.1463e-07, 5.8058e-07, 8.5707e-07, 8.0941e-07, 4.2924e-07, 2.7183e-07,
         1.5320e-07, 8.4392e-08, 5.4033e-0

In [ ]:
summarize_results(load_experiment_log())[['dt_created', 'num_epochs', 'learning_rate', 'clip_grad_max_norm', 'val_loss']].head()

In [ ]:
plot_single_learning_curve(results)

In [ ]:
# Epoch: 199.00, Train Loss: 0.32, Val Loss: 13.19, Train BLEU: 98.94, Val BLEU: 0.27
plot_single_learning_curve(results)

In [ ]:
# with attention energies = v_broadcast.bmm(torch.tanh(self.attn(concat)).transpose(1, 2)) # switched order  
# Epoch: 199.00, Train Loss: 0.63, Val Loss: 12.82, Train BLEU: 92.05, Val BLEU: 0.38
plot_single_learning_curve(results)

In [ ]:
for i, token in enumerate(vocab[SRC_LANG]['id2token']): 
    if i < 20: 
        print("{}: {}".format(i, token))

In [ ]:
for i, token in enumerate(vocab[TARG_LANG]['id2token']): 
    if i < 20: 
        print("{}: {}".format(i, token))

In [ ]:
import torch
x = torch.arange(0, 3*5*10).view(3, 5, 10)
print(x)
y = x[1:, :, :]
print(y)
z = y.view(-1, 10)
print(z)

In [ ]:
t = torch.arange(0, 2*5).view(5, 2)
print(t)
u = t.contiguous().view(-1)
print(u)
v = t.permute(1, 0)
print(v)
w = v.contiguous().view(-1)
print(w)

In [ ]:
a = torch.arange(0, 2*1*300)
print(a)
b = a.view(-1, 1, 300)
print(b.size())

In [ ]:
for i, (src_idxs, targ_idxs, src_lens, targ_lens) in enumerate(full_loaders['train']):
#     print(i)
#     print(src_idxs.size())
#     print(src_idxs)
#     print(src_lens)
#     print(targ_idxs.size())
#     print(targ_idxs)
#     print(targ_lens)
    id2token = vocab[SRC_LANG]['id2token']
    test_tensor = src_idxs
    list_of_lists = test_tensor.numpy().astype(int).tolist()
    to_token = lambda l: ' '.join([id2token[idx] for idx in l])
    list_of_lists_tokens = [to_token(l) for l in list_of_lists] 
    break 